In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "{language}에 대한 시를 써주세요"),
    ]
)

poet_chain = poet_prompt | chat

In [5]:
analyze_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 시를 분석하는 분석가 입니다.",
        ),
        ("human", "{poet}"),
    ]
)


ana_chain = analyze_prompt | chat


final_chain = {"poet": poet_chain} | ana_chain

final_chain.invoke({"language": "python"})

파이썬 코드를 작성하며
세상을 탐험하는 모험가
간결하고 강력한 힘을 지닌
파이썬, 나의 친구여 영원히 함께하리.주어진 시는 파이썬 프로그래밍 언어에 대한 사랑과 그것이 주는 힘에 대한 내용을 담고 있습니다. 시인은 파이썬 코드를 작성하며 세상을 탐험하는 것을 모험가의 활동에 비유하여 표현하고 있습니다. 또한 파이썬을 간결하고 강력한 힘을 지닌 친구로 묘사하여, 파이썬에 대한 애정과 신뢰를 드러내고 있습니다. 마지막으로 "영원히 함께하리"라는 구절은 시인이 파이썬과의 관계를 오랫동안 유지하고자 하는 강한 의지를 나타내고 있습니다.

AIMessageChunk(content='주어진 시는 파이썬 프로그래밍 언어에 대한 사랑과 그것이 주는 힘에 대한 내용을 담고 있습니다. 시인은 파이썬 코드를 작성하며 세상을 탐험하는 것을 모험가의 활동에 비유하여 표현하고 있습니다. 또한 파이썬을 간결하고 강력한 힘을 지닌 친구로 묘사하여, 파이썬에 대한 애정과 신뢰를 드러내고 있습니다. 마지막으로 "영원히 함께하리"라는 구절은 시인이 파이썬과의 관계를 오랫동안 유지하고자 하는 강한 의지를 나타내고 있습니다.')